In [ ]:
#！/usr/bin/python
# -*- coding: utf-8 -*-

In [ ]:
import pyodbc
import textwrap
import pandas as pd

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

In [ ]:
'''
Seasoning Trends: level='quarter'
Week Trends: level='weekday'
Time Trends: level='hour'
'''

raw_sql = textwrap.dedent("""
    SET DATEFIRST 1;
    SELECT DATEPART({level},r.timestamp1) AS {level}
          ,s.category AS category
          ,COUNT(r.url) AS [Number of visits]
          ,COUNT(DISTINCT r.userid) AS [Unique Visitors]
    FROM [data].[dbo].[health_records] r
    JOIN [data].[dbo].[healthsites] s
    ON r.website = s.abbreviation
    GROUP BY DATEPART({level},r.timestamp1), s.category
    ORDER BY {level}, category

""")

In [ ]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=localhost;DATABASE=data;')

cursor = cnxn.cursor()

In [ ]:
color_picker = {
    'E-pharmacy': 'rgb(55, 83, 109)',
    'Lifestyle': 'rgb(22, 96, 167)',
    'Medical': 'rgb(205, 12, 24)',
}

In [ ]:
title_switcher = {
    'quarter': 'Seasoning Trends',
    'weekday': 'Week Trends',
    'hour': 'Time Trends',
}

In [ ]:
plotly.tools.set_credentials_file(username='tenaghirmb', api_key='please_use_your_own_api')

In [ ]:
plotly.offline.init_notebook_mode(connected=True)

In [ ]:
def to_df(level):
    sql = raw_sql.format(level=level)
    rows = cursor.execute(sql).fetchall()
    df = pd.DataFrame( [[ij for ij in i] for i in rows] )
    df.rename(columns={0: level, 1:'category', 2: 'Number of visits', 3:'Unique Visitors'}, inplace=True)
    return df

In [ ]:
def make_trace(df, category, target):
    is_category = df['category'] == category
    return go.Scatter(
        x = df[is_category].iloc[:, 0],
        y = df[is_category][target],
        name = category,
        mode = 'lines+markers',
        line = dict(
            color = color_picker.get(category),
            width = 3,
        )
    )

In [ ]:
def make_charts(level, target):
    df = to_df(level)
    Epharmacy = make_trace(df, 'E-pharmacy', target)
    Lifestyle = make_trace(df, 'Lifestyle', target)
    Medical = make_trace(df, 'Medical', target)
    data = [Medical, Lifestyle, Epharmacy]
    layout = go.Layout(
        xaxis=dict(
            title=level,
            titlefont=dict(
                size=16,
            ),
            tickfont=dict(
                size=14,
            ),
            nticks=24,
            type='category',
        ),
        yaxis=dict(
            title=target,
            titlefont=dict(
                size=16,
            ),
            tickfont=dict(
                size=14,
            )
        ),
        legend=dict(
            x=1.0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename=title_switcher.get(level)+' - '+target)
    #py.iplot(fig, filename=title_switcher.get(level)+' - '+target)

In [ ]:
make_charts('quarter', 'Number of visits')
make_charts('quarter', 'Unique Visitors')
make_charts('weekday', 'Number of visits')
make_charts('weekday', 'Unique Visitors')
make_charts('hour', 'Number of visits')
make_charts('hour', 'Unique Visitors')